Improved-LLM-Tutor

In [0]:
# Standard library imports
import os
import time
import json
from typing import Dict, List, Any, Optional, Union, Callable

# Third-party imports
from dotenv import load_dotenv
from IPython.display import Markdown, display, HTML, update_display
from openai import OpenAI
import ollama
import pandas as pd
import matplotlib.pyplot as plt

# Try to import rich, install if not available
try:
    from rich.console import Console
    from rich.markdown import Markdown as RichMarkdown
    from rich.panel import Panel
except ImportError:
    !pip install rich
    from rich.console import Console
    from rich.markdown import Markdown as RichMarkdown
    from rich.panel import Panel


In [0]:

# Constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
DEFAULT_SYSTEM_PROMPT = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"

# Set up environment
load_dotenv()
openai = OpenAI()
console = Console()


In [0]:

class LLMTutor:
    """
    A class that provides tutoring functionality using multiple LLM models.
    """
    
    def __init__(self, 
                 system_prompt: str = DEFAULT_SYSTEM_PROMPT,
                 gpt_model: str = MODEL_GPT,
                 llama_model: str = MODEL_LLAMA):
        """
        Initialize the LLM Tutor with specified models and system prompt.
        
        Args:
            system_prompt: The system prompt to use for the LLMs
            gpt_model: The OpenAI GPT model to use
            llama_model: The Ollama model to use
        """
        self.system_prompt = system_prompt
        self.gpt_model = gpt_model
        self.llama_model = llama_model
        self.history: List[Dict[str, Any]] = []
        self.response_times = {'gpt': [], 'llama': []}
        
    def format_question(self, question: str) -> str:
        """
        Format the user's question with a standard prefix.
        
        Args:
            question: The user's question
            
        Returns:
            Formatted question with prefix
        """
        return f"Please give a detailed explanation to the following question: {question}"
    
    def create_messages(self, question: str) -> List[Dict[str, str]]:
        """
        Create the message structure for LLM API calls.
        
        Args:
            question: The user's question
            
        Returns:
            List of message dictionaries
        """
        formatted_question = self.format_question(question)
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": formatted_question}
        ]
    
    def get_gpt_response(self, 
                        question: str, 
                        stream: bool = True) -> str:
        """
        Get a response from the GPT model.
        
        Args:
            question: The user's question
            stream: Whether to stream the response
            
        Returns:
            The model's response as a string
        """
        messages = self.create_messages(question)
        start_time = time.time()
        
        try:
            if stream:
                return self._stream_gpt_response(messages)
            else:
                response = openai.chat.completions.create(
                    model=self.gpt_model, 
                    messages=messages
                )
                elapsed = time.time() - start_time
                self.response_times['gpt'].append(elapsed)
                return response.choices[0].message.content
        except Exception as e:
            console.print(f"[bold red]Error with GPT model:[/bold red] {str(e)}")
            return f"Error: {str(e)}"
    
    def _stream_gpt_response(self, messages: List[Dict[str, str]]) -> str:
        """
        Stream a response from the GPT model.
        
        Args:
            messages: The messages to send to the model
            
        Returns:
            The complete response as a string
        """
        start_time = time.time()
        try:
            stream = openai.chat.completions.create(
                model=self.gpt_model, 
                messages=messages,
                stream=True
            )
            
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            
            for chunk in stream:
                delta_content = chunk.choices[0].delta.content or ''
                response += delta_content
                # Clean the response for display
                clean_response = response.replace("```python", "```").replace("```", "")
                update_display(Markdown(clean_response), display_id=display_handle.display_id)
            
            elapsed = time.time() - start_time
            self.response_times['gpt'].append(elapsed)
            return response
        except Exception as e:
            console.print(f"[bold red]Error streaming GPT response:[/bold red] {str(e)}")
            return f"Error: {str(e)}"
    
    def get_llama_response(self, question: str) -> str:
        """
        Get a response from the Llama model.
        
        Args:
            question: The user's question
            
        Returns:
            The model's response as a string
        """
        messages = self.create_messages(question)
        start_time = time.time()
        
        try:
            response = ollama.chat(model=self.llama_model, messages=messages)
            elapsed = time.time() - start_time
            self.response_times['llama'].append(elapsed)
            return response['message']['content']
        except Exception as e:
            console.print(f"[bold red]Error with Llama model:[/bold red] {str(e)}")
            return f"Error: {str(e)}"
    
    def ask(self, question: str, models: List[str] = ['gpt', 'llama']) -> Dict[str, str]:
        """
        Ask a question to one or more models.
        
        Args:
            question: The user's question
            models: List of models to query ('gpt', 'llama', or both)
            
        Returns:
            Dictionary with model responses
        """
        responses = {}
        
        # Store the question in history
        self.history.append({
            'question': question,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
            'responses': {}
        })
        
        # Get responses from requested models
        if 'gpt' in models:
            console.print(Panel(f"[bold blue]Getting response from {self.gpt_model}...[/bold blue]"))
            gpt_response = self.get_gpt_response(question)
            responses['gpt'] = gpt_response
            self.history[-1]['responses']['gpt'] = gpt_response
            
        if 'llama' in models:
            console.print(Panel(f"[bold green]Getting response from {self.llama_model}...[/bold green]"))
            llama_response = self.get_llama_response(question)
            responses['llama'] = llama_response
            self.history[-1]['responses']['llama'] = llama_response
            display(Markdown(f"## {self.llama_model} Response\n{llama_response}"))
            
        return responses
    
    def compare_responses(self, question: str = None) -> None:
        """
        Compare responses from different models side by side.
        
        Args:
            question: Optional specific question to compare responses for
        """
        if question:
            responses = self.ask(question)
        else:
            # Use the most recent question from history
            if not self.history:
                console.print("[bold red]No questions in history to compare[/bold red]")
                return
            responses = self.history[-1]['responses']
            question = self.history[-1]['question']
        
        # Create HTML for side-by-side comparison
        html = f"""
        <div style="display: flex; width: 100%;">
            <div style="flex: 1; padding: 10px; border: 1px solid #ddd; border-radius: 5px; margin-right: 10px;">
                <h3 style="color: #4285F4;">{self.gpt_model}</h3>
                <div style="white-space: pre-wrap;">{responses.get('gpt', 'No response')}</div>
            </div>
            <div style="flex: 1; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                <h3 style="color: #34A853;">{self.llama_model}</h3>
                <div style="white-space: pre-wrap;">{responses.get('llama', 'No response')}</div>
            </div>
        </div>
        """
        display(HTML(html))
    
    def show_performance_metrics(self) -> None:
        """
        Display performance metrics for the models.
        """
        if not self.response_times['gpt'] and not self.response_times['llama']:
            console.print("[bold yellow]No performance data available yet[/bold yellow]")
            return
        
        # Create DataFrame for metrics
        data = {
            'Model': [],
            'Response Time (s)': []
        }
        
        for model, times in self.response_times.items():
            for t in times:
                data['Model'].append(model)
                data['Response Time (s)'].append(t)
        
        df = pd.DataFrame(data)
        
        # Calculate statistics
        stats = df.groupby('Model')['Response Time (s)'].agg(['mean', 'min', 'max', 'count'])
        
        # Display statistics
        console.print("\n[bold]Performance Statistics:[/bold]")
        console.print(stats)
        
        # Create visualization
        plt.figure(figsize=(10, 6))
        
        # Box plot
        ax = plt.subplot(1, 2, 1)
        df.boxplot(column='Response Time (s)', by='Model', ax=ax)
        plt.title('Response Time Distribution')
        plt.suptitle('')
        
        # Bar chart for average times
        ax = plt.subplot(1, 2, 2)
        stats['mean'].plot(kind='bar', ax=ax, color=['#4285F4', '#34A853'])
        plt.title('Average Response Time')
        plt.ylabel('Seconds')
        
        plt.tight_layout()
        plt.show()
    
    def save_history(self, filename: str = 'tutor_history.json') -> None:
        """
        Save the question and response history to a file.
        
        Args:
            filename: The filename to save to
        """
        try:
            with open(filename, 'w') as f:
                json.dump(self.history, f, indent=2)
            console.print(f"[bold green]History saved to {filename}[/bold green]")
        except Exception as e:
            console.print(f"[bold red]Error saving history:[/bold red] {str(e)}")
    
    def load_history(self, filename: str = 'tutor_history.json') -> None:
        """
        Load question and response history from a file.
        
        Args:
            filename: The filename to load from
        """
        try:
            with open(filename, 'r') as f:
                self.history = json.load(f)
            console.print(f"[bold green]History loaded from {filename}[/bold green]")
        except FileNotFoundError:
            console.print(f"[bold yellow]History file {filename} not found[/bold yellow]")
        except Exception as e:
            console.print(f"[bold red]Error loading history:[/bold red] {str(e)}")


In [0]:

# Create a tutor instance
tutor = LLMTutor()
console.print("[bold green]LLM Tutor initialized successfully![/bold green]")


In [0]:

# Define your question here
question = """
Given a list of dictionaries called 'books', write code to find and print all information 
about the book titled 'Mastery' by Robert Greene.
"""

console.print(Panel(f"[bold]Question:[/bold]\n{question}", border_style="blue"))


In [0]:

# Get responses from both models
responses = tutor.ask(question)


In [0]:

# Compare responses side by side
tutor.compare_responses()


In [0]:

# Show performance metrics
tutor.show_performance_metrics()


In [0]:

# Save history to a file
tutor.save_history("my_tutor_session.json")


In [0]:
# Define a new question
new_question = "Explain how to implement a binary search algorithm in Python."

console.print(Panel(f"[bold]New Question:[/bold]\n{new_question}", border_style="green"))

# Get responses for the new question
new_responses = tutor.ask(new_question)

# Compare responses
tutor.compare_responses()
